In [1]:
%cd ..
import os
import time

/data/apperception/apperception


In [2]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F
from apperception.predicate import camera, objects
from optimized_ingestion.utils.preprocess import preprocess
database.connection
from optimized_ingestion.cache import disable_cache
disable_cache()

/home/local/apperception/.installs/mambaforge/envs/apperception/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
YOLOv5 🚀 2022-11-23 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)



Using cuda:0


Using cache found in /data/apperception/apperception/weights/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-30 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
NUSCENES_PROCESSED_DATA_FULL = "NUSCENES_PROCESSED_DATA_FULL"
if NUSCENES_PROCESSED_DATA_FULL in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA_FULL]
else:
    DATA_DIR = "/data/apperception-data/processed/nuscenes/full-dataset-v1.0/Trainval"
NUSCENES_RAW_DATA_FULL = "NUSCENES_RAW_DATA_FULL"
if NUSCENES_RAW_DATA_FULL in os.environ:
    RAW_DATA_DIR = os.environ[NUSCENES_RAW_DATA_FULL]
else:
    RAW_DATA_DIR = "/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Trainval"

In [4]:
import cv2
def construct_video(frames, size=(1600, 900), base=False, vid_name=None):
    unique_frames = []
    for f in frames:
        if f not in unique_frames:
            unique_frames.append(f)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vid_name = "./outputs/fig14_"+vid_name if vid_name else f"./outputs/{'fig14_base' if base else 'fig14_optimized'}.avi"
    display_video = cv2.VideoWriter(vid_name,fourcc, 1, size)
    for frame in unique_frames:
        img_path = os.path.join(RAW_DATA_DIR, frame)
        img = cv2.imread(img_path)
        display_video.write(img)

    display_video.release()

In [5]:
name = 'ScenicWorld' # world name
world = empty_world(name=name)

obj1 = objects[0]
cam = camera
world = world.filter(
    (F.like(obj1.type, 'car') | F.like(obj1.type, 'truck') | F.like(obj1.type, 'bus')) &
    (F.distance(cam.ego, obj1.trans@cam.timestamp) < 50) &
    (F.view_angle(obj1.trans@cam.time, cam.ego) < 70 / 2) &
    F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.ego)), -180, -90) &
    F.contained(cam.ego, F.road_segment('road')) &
    F.contained(obj1.trans@cam.time, F.road_segment('road')) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, F.road_direction(obj1.traj@cam.time, cam.ego)), -15, 15) &
    (F.distance(cam.ego, obj1.trans@cam.time) < 10)
)

In [ ]:
preprocess(world, DATA_DIR, base=False, benchmark_path="./outputs/fig14_full_optimization.txt")

YOLOv5 🚀 2022-11-23 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)

Using cache found in /data/apperception/weights/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-30 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


scene-0062-CAM_FRONT --------------------------------------------------------------------------------
Stage:  DecodeFrame.ParallelDecodeFrame


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:03<00:00, 24.40it/s]


None
228
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection2D.YoloDetection


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 228/228 [00:02<00:00, 107.21it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 227/227 [00:02<00:00, 111.63it/s]


None
227
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection2D.ObjectTypeFilter
None
227
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection3D.From2DAndRoad


227it [00:00, 3161.93it/s]


None
227
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  DetectionEstimation
ego_speed:  5.313405946359398


 24%|██████████████████████████████████████▋                                                                                                                           | 54/226 [00:01<00:08, 20.99it/s]

In [8]:
import shapely
extend_line = shapely.geometry.LineString([(1190.2551876969633,921.7528591561929), (1190.9155903849717,922.50377079525)])
polygon_points = list(map(shapely.geometry.Point, [(1218.119877490335, 950.0193724149779), (1218.0194019524067, 949.7184794265356), (1217.7629714489142, 947.9125798056587), (1214.2723391730558, 943.6692407213698), (1187.9419320255877, 929.5604088522862), (1188.5609617879406, 929.8039367181402), (1211.1843021612417, 955.7313308551293), (1211.4738751317575, 956.048997716631), (1213.8033793080144, 953.7450882131476), (1216.2430133445678, 951.4612151907936), (1217.429678325924, 950.4265496073891), (1218.119877490335, 950.0193724149779)]))
polygon = shapely.geometry.Polygon(polygon_points)
polygon.contains(shapely.geometry.Point((1185.38623046875, 919.1126708984375)))

False

In [8]:
optimized_query_start = time.time()
id_time_camId_filename = world.get_id_time_camId_filename(1)
print(f'optimized query time: {time.time()-optimized_query_start}')

execute: filter
execute: get_id_time_camId_filename
get_id_time_camId_filename 
        SELECT t0.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as t0
        
        JOIN Cameras USING (cameraId)
        WHERE ((t0.objectType LIKE 'car' OR t0.objectType LIKE 'truck' OR t0.objectType LIKE 'bus') AND (ST_Distance(egoTranslation,valueAtTimestamp(t0.translations,timestamp))<50) AND (viewAngle(valueAtTimestamp(t0.translations,timestamp),egoHeading,egoTranslation)<35.0) AND angleBetween(facingRelative(egoHeading,roadDirection(egoTranslation,egoHeading)),-180,-90) AND contained(egoTranslation,roadSegment('road')) AND contained(valueAtTimestamp(t0.translations,timestamp),roadSegment('road')) AND angleBetween(facingRelative((headingAtTimestamp(t0.itemHeadings, timestamp))::real,roadDirection(valueAtTimestamp(t0.trajCentroids,timestamp),egoHeading)),-15,15) AND (ST_Distance(egoTranslation,valueAtTimestamp(t0.translations,times

In [9]:
id_time_camId_filename

[('scene-0769-CAM_FRONT_obj_122',
  datetime.datetime(2018, 8, 30, 12, 31, 30, 862404, tzinfo=datetime.timezone.utc),
  'scene-0769-CAM_FRONT',
  'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657490862404.jpg'),
 ('scene-0769-CAM_FRONT_obj_122',
  datetime.datetime(2018, 8, 30, 12, 31, 30, 912404, tzinfo=datetime.timezone.utc),
  'scene-0769-CAM_FRONT',
  'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657490912404.jpg'),
 ('scene-0769-CAM_FRONT_obj_134',
  datetime.datetime(2018, 8, 30, 12, 31, 30, 912404, tzinfo=datetime.timezone.utc),
  'scene-0769-CAM_FRONT',
  'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657490912404.jpg'),
 ('scene-0769-CAM_FRONT_obj_122',
  datetime.datetime(2018, 8, 30, 12, 31, 31, 12404, tzinfo=datetime.timezone.utc),
  'scene-0769-CAM_FRONT',
  'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491012404.jpg'),
 ('scene-0769-CAM_FRONT_obj_122',
  datetime.datetime(2018, 8, 30, 12, 31, 31, 11

In [10]:
optimized_results = [e[3] for e in id_time_camId_filename]
construct_video(optimized_results)

In [11]:
preprocess(world, DATA_DIR, ['scene-0769-CAM_FRONT'])


YOLOv5 🚀 2022-11-23 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)

Using cache found in /data/apperception/weights/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-30 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


scene-0769-CAM_FRONT --------------------------------------------------------------------------------
Stage:  DecodeFrame.ParallelDecodeFrame


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:04<00:00, 19.67it/s]


None
239
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection2D.YoloDetection


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 239/239 [00:02<00:00, 106.27it/s]


...

None
239
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Tracking2D.StrongSORT
Successfully loaded pretrained weights from "/data/apperception/apperception/weights/osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 239/239 [00:17<00:00, 13.77it/s]


None
239
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Tracking3D.From2DAndRoad


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 239/239 [00:00<00:00, 7224.99it/s]


None
239
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  SegmentTrajectory.FromTracking3D
None
239
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Time taken to insert trajectories: 0.30182981491088867
info found []
total preprocess time 36.29892897605896


In [12]:
base_query_start = time.time()
id_time_camId_filename_base = world.get_id_time_camId_filename(1)
print(f'base query time: {time.time()-base_query_start}')

execute: filter
execute: get_id_time_camId_filename
get_id_time_camId_filename 
        SELECT t0.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as t0
        
        JOIN Cameras USING (cameraId)
        WHERE ((t0.objectType LIKE 'car' OR t0.objectType LIKE 'truck' OR t0.objectType LIKE 'bus') AND (ST_Distance(egoTranslation,valueAtTimestamp(t0.translations,timestamp))<50) AND (viewAngle(valueAtTimestamp(t0.translations,timestamp),egoHeading,egoTranslation)<35.0) AND angleBetween(facingRelative(egoHeading,roadDirection(egoTranslation,egoHeading)),-180,-90) AND contained(egoTranslation,roadSegment('road')) AND contained(valueAtTimestamp(t0.translations,timestamp),roadSegment('road')) AND angleBetween(facingRelative((headingAtTimestamp(t0.itemHeadings, timestamp))::real,roadDirection(valueAtTimestamp(t0.trajCentroids,timestamp),egoHeading)),-15,15) AND (ST_Distance(egoTranslation,valueAtTimestamp(t0.translations,times

In [13]:
id_time_camId_filename_base

[('scene-0769-CAM_FRONT_obj_194',
  datetime.datetime(2018, 8, 30, 12, 31, 30, 862404, tzinfo=datetime.timezone.utc),
  'scene-0769-CAM_FRONT',
  'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657490862404.jpg'),
 ('scene-0769-CAM_FRONT_obj_194',
  datetime.datetime(2018, 8, 30, 12, 31, 30, 912404, tzinfo=datetime.timezone.utc),
  'scene-0769-CAM_FRONT',
  'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657490912404.jpg'),
 ('scene-0769-CAM_FRONT_obj_217',
  datetime.datetime(2018, 8, 30, 12, 31, 30, 912404, tzinfo=datetime.timezone.utc),
  'scene-0769-CAM_FRONT',
  'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657490912404.jpg'),
 ('scene-0769-CAM_FRONT_obj_194',
  datetime.datetime(2018, 8, 30, 12, 31, 31, 12404, tzinfo=datetime.timezone.utc),
  'scene-0769-CAM_FRONT',
  'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491012404.jpg'),
 ('scene-0769-CAM_FRONT_obj_194',
  datetime.datetime(2018, 8, 30, 12, 31, 31, 11

In [14]:
base_results = [e[3] for e in id_time_camId_filename_base]
construct_video(base_results, base=True)

In [15]:
base_results

['sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657490862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657490912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657490912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491012404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491112404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491162404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__15356574914124

In [16]:
false_negative = [f for f in base_results if f not in optimized_results]
print(len(false_negative))
construct_video(false_negative, vid_name="false_negative.avi")

0


In [17]:
false_positive = [f for f in optimized_results if f not in base_results]
print(len(false_positive))
construct_video(false_positive, vid_name="false_positive.avi")

0


In [18]:
investigation = [[217, 1205.0, 198.0, 391.0, 525.0], [104, 1198.0, 192.0, 390.0, 520.0]]

In [19]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

im = Image.open('/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Trainval/sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657490862404.jpg')

# Create figure and axes
fig, ax = plt.subplots()

# Display the image
ax.imshow(im)

# Create a Rectangle patch
for obj in investigation:
    obj_id = obj[0]
    x,y,w,h = obj[1:5]
    plt.text(x, y+15, 'obj'+str(obj_id))
    rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')

    # Add the patch to the Axes
    ax.add_patch(rect)

plt.show()